In [1]:
# open data  

# Yale 교수 전공 및 general 모든 정보 link

In [2]:
import os
import numpy as np
import pandas as pd
from IPython import display

pd.options.display.max_rows = 999 # 데이터 프레임 표시 최대 열수를 999로 지정
pd.set_option('display.max_columns',999) # 데이터 프레임 표시 최대 행수를 999로 지정


In [9]:
# get excel data to dataframe
file_name = './college_datasets/Yale/Yale 교수 전공 및 general 모든 정보 link.csv'
df = pd.read_csv(file_name)
df

,major_title,info_type,Link,Unnamed: 3
0,African American Studies,general_keywords,https://afamstudies.yale.edu/,NaN
1,African American Studies,general_keywords,https://afamstudies.yale.edu/about-us,NaN
2,African American Studies,facilities_resources,https://yalecollege.yale.edu/communities/cultu...,NaN
3,African American Studies,courses_concentrations,https://afamstudies.yale.edu/undergraduate-major,NaN
4,African American Studies,facilities_resources,http://catalog.yale.edu/ycps/subjects-of-instr...,NaN
...,...,...,...,...
2947,"Women's, Gender and Sexuality Studies",professors,https://wgss.yale.edu/people/eda-pepi,NaN
2948,"Women's, Gender and Sexuality Studies",professors,https://wgss.yale.edu/people/graeme-reid,NaN
2949,"Women's, Gender and Sexuality Studies",professors,https://wgss.yale.edu/people/evren-savci,NaN
2950,"Women's, Gender and Sexuality Studies",professors,https://wgss.yale.edu/people/maria-trumpler,NaN


In [72]:
len(df.index)

2952

In [11]:
# major_title + info_type 으로 파일명을 만들고 링크타고 데이터 추출후 저장

In [21]:
mjr_title = df.iloc[0]['major_title']
mjr_title 

'African American Studies'

In [22]:
info_type = df.iloc[0]['info_type']
info_type

'general_keywords'

In [23]:
link = df.iloc[0]['Link']
link

'https://afamstudies.yale.edu/'

In [34]:
mk_file_full_name = "Yale" + "_" +  mjr_title + "_" + info_type + "_info" + ".txt"

In [36]:
# dataframe에서 데이터 추출하고 파일명으로 저장하기
md_file = open(mk_file_full_name, 'w')
# Yale_African American Studies_general_keywords_info.txt
md_fine.close()

In [37]:
# 링크타고 들어가서 키워드 크롤링
# 검색어로 크롬 실행, 실행결과의 링크 추출, 링크를 타고 각 페이지 접속, 페이지 내용 전체 크롤링, text 추출하고, 단어 리스트로 만드는 코드

import re
import requests
import pandas as pd
import openpyxl
from urllib.parse import ParseResultBytes, quote_plus
from bs4 import BeautifulSoup
from selenium import webdriver

from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

from nltk.corpus import stopwords
stop = stopwords.words('english')
stop_words = set(stopwords.words('english')) 


options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('window-size=1920x1080')
options.add_argument("disable-gpu")
# 혹은 options.add_argument("--disable-gpu")

# UserAgent값을 바꿔줍시다! 서버가 인식하지 못하도록 가상으로 headless 값 추가함ㅠ
options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36")


In [53]:
def linked_search_result(linked_page):
#     baseUrl = 'https://www.google.com/search?q='

#     #plusUrl = input('무엇을 검색할까요? :')
#     plusUrl = input_word

#     # url = baseUrl + quote_plus(plusUrl)
#     url = baseUrl + plusUrl
#     # 한글을 사용할 경우 :  quote_plus 적용 - URL에 막 %CE%GD%EC 이런 거 생성해줌

    driver = webdriver.Chrome(executable_path= r'./data/chromedriver_mac_ver_90', chrome_options=options)
    driver.get(linked_page)

#     html = driver.page_source
#     soup = BeautifulSoup(html, features="html.parser")

#     v = soup.select('.yuRUbf')

#     search_title_result = []
#     search_linked_contents_result = []
#     for i in v:
#         #print(i.select_one('.LC20lb.DKV0Md').text)
#         search_title_result.append(i.select_one('.LC20lb.DKV0Md').text)
#         #print(i.a.attrs['href'])
#         search_linked_contents_result.append(i.a.attrs['href'])
#         #print()


    # search_linked_contents_result 의 각 링크로 접속하여 해당 내용을 모두 text로 크롤링한 후, 단어만 추리고, 다시 주요키워드를 추출한다.
    
    get_all_linked_web_data = []
    
    driver.get(linked_page)
    html = driver.page_source
    get_all_data = BeautifulSoup(html, features="html.parser")
    get_all_linked_web_data.append(get_all_data)
        
    body = re.search('<body.*/body>', html, re.I|re.S)
    if (body is None):
        print ("No <body> in html")
        exit()
            
    body = body.group()
    #print(body)
    
    # 추출된 정보 클린징
    korean = re.compile('[\u3131-\u3163\uac00-\ud7a3]+')#한글제거

    item_extract = str(body).replace('\n', ' ')
    item_extract = re.sub('<span.*?>.*?</span>', ' ', item_extract)
    item_extract = re.sub('<b>.*?</b>', ' ', item_extract)    
    item_extract = re.sub('<.*?>', ' ', item_extract)        
    item_extract = item_extract.replace('\t', ' ')
    item_extract = re.sub(korean, '', item_extract)
    item_extract = re.sub('[-=.#/?:$}]', ' ', item_extract)
    item_extract = re.sub("[-=+,#/\?:^$.@*\"※~&%ㆍ!』;{}()'\\‘|\(\)\[\]\<\>`\'…》]", ' ', item_extract)
    #print (item_extract)
    
    driver.close()

    return item_extract



In [54]:
get_result = linked_search_result('https://afamstudies.yale.edu/') 

/Users/kimkwangil/opt/anaconda3/envs/py37pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: use options instead of chrome_options
  # This is added back by InteractiveShellApp.init_path()


In [55]:
get_result

'    omega media query dummy   position  relative  z index   1       omega media query dummy   z index  2       omega media query dummy   z index  0      omega media query dummy   z index  1      omega media query dummy   z index  2               Skip to main content                                                                                                                                                                                                                                                                                                                                                                                                                 Department of African American Studies nbsp                                                                                                                                                                         Graduate Program     Undergraduate Major     Research  amp  Collections     Media Gallery     People     Contact Us     

## test code

In [59]:
import os
import numpy as np
import pandas as pd
from IPython import display

pd.options.display.max_rows = 999 # 데이터 프레임 표시 최대 열수를 999로 지정
pd.set_option('display.max_columns',999) # 데이터 프레임 표시 최대 행수를 999로 지정

import re
import requests
import openpyxl
from urllib.parse import ParseResultBytes, quote_plus
from bs4 import BeautifulSoup
from selenium import webdriver

from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

from nltk.corpus import stopwords
stop = stopwords.words('english')
stop_words = set(stopwords.words('english')) 


options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('window-size=1920x1080')
options.add_argument("disable-gpu")
# 혹은 options.add_argument("--disable-gpu")

# UserAgent값을 바꿔줍시다! 서버가 인식하지 못하도록 가상으로 headless 값 추가함ㅠ
options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36")



In [60]:
def cleaning_data(input_data):
    remove = input_data.replace("  "," ") # 변환
    remove_ = re.sub(r"\t", " ", remove) # 제거
    remove__ = re.sub(r"\n", " ", remove_) # 제거
    remove__ = remove__.replace("   ", " ")
    remove__ = remove__.replace("  ", " ")
    remove__ = remove__.replace(" ", ",")
    remove__ = remove__.replace("…/", " ")
    remove__ = remove__.replace("…", " ")
    remove__ = remove__.replace("/", " ")
    remove__ = remove__.replace(" ", ",")
    remove__ = remove__.replace(")", ",")
    remove__ = remove__.replace("(", ",")
    preprossed = remove__.split(",") # 단어를 리스트로 변환
    #print(preprossed)
    
    # 표제어 추출, 동사는 현재형으로 변환
    lemma_list =[]
    for i in preprossed:
        lema_re = lemmatizer.lemmatize(i, pos='v') #표제어 추출, 동사는 현재형으로 변환
        lemma_list.append(lema_re)
    
    # 표제어 추출
    ext_lema = [lemmatizer.lemmatize(w) for w in preprossed]
    # 중복값을 제거하고
    rm_dupli = set(ext_lema)
    # 다시 리스트로 만들고
    re_li = list(rm_dupli)
    # 빈 값은 제거하고
    get_wd =list(filter(None, re_li))
    # 소문자로 모두 변환
    lower_wd = [i.lower() for i in get_wd]
    
    result = []
    for w in lower_wd: 
        if w not in stop_words: 
            result.append(w)
    return result

In [82]:
def linked_search_result(linked_page):
#     baseUrl = 'https://www.google.com/search?q='

#     #plusUrl = input('무엇을 검색할까요? :')
#     plusUrl = input_word

#     # url = baseUrl + quote_plus(plusUrl)
#     url = baseUrl + plusUrl
#     # 한글을 사용할 경우 :  quote_plus 적용 - URL에 막 %CE%GD%EC 이런 거 생성해줌

    driver = webdriver.Chrome(executable_path= r'./data/chromedriver_mac_ver_90', chrome_options=options)
    
    try:
        driver.get(linked_page)
        
    #     html = driver.page_source
    #     soup = BeautifulSoup(html, features="html.parser")

    #     v = soup.select('.yuRUbf')

    #     search_title_result = []
    #     search_linked_contents_result = []
    #     for i in v:
    #         #print(i.select_one('.LC20lb.DKV0Md').text)
    #         search_title_result.append(i.select_one('.LC20lb.DKV0Md').text)
    #         #print(i.a.attrs['href'])
    #         search_linked_contents_result.append(i.a.attrs['href'])
    #         #print()


        # search_linked_contents_result 의 각 링크로 접속하여 해당 내용을 모두 text로 크롤링한 후, 단어만 추리고, 다시 주요키워드를 추출한다.

        get_all_linked_web_data = []

        driver.get(linked_page)
        html = driver.page_source
        get_all_data = BeautifulSoup(html, features="html.parser")
        get_all_linked_web_data.append(get_all_data)

        body = re.search('<body.*/body>', html, re.I|re.S)
        if (body is None):
            print ("No <body> in html")
            exit()

        body = body.group()
        #print(body)

        # 추출된 정보 클린징
        korean = re.compile('[\u3131-\u3163\uac00-\ud7a3]+')#한글제거

        item_extract = str(body).replace('\n', ' ')
        item_extract = re.sub('<span.*?>.*?</span>', ' ', item_extract)
        item_extract = re.sub('<b>.*?</b>', ' ', item_extract)    
        item_extract = re.sub('<.*?>', ' ', item_extract)        
        item_extract = item_extract.replace('\t', ' ')
        item_extract = re.sub(korean, '', item_extract)
        item_extract = re.sub('[-=.#/?:$}]', ' ', item_extract)
        item_extract = re.sub("[-=+,#/\?:^$.@*\"※~&%ㆍ!』;{}()'\\‘|\(\)\[\]\<\>`\'…》]", ' ', item_extract)
        #print (item_extract)

        driver.close()

    except:
        item_extract = ["not enough infomation"]

    return item_extract



In [68]:
# 자동으로 데이터 추출코드, 1)파일명 생성 2)링크접속 3)데이터크롤링 4)키워드로 저장
# file_name : 추출해야할 csv 파일 경로 및 파일명 지정
#     file_name = './college_datasets/Yale/Yale 교수 전공 및 general 모든 정보 link.csv'

# college_name : 'Yale'
# file_name =: './college_datasets/Yale/Yale 교수 전공 및 general 모든 정보 link.csv'
def input_name_of_college(college_name, file_name):
    # import file to dataframe
    df = pd.read_csv(file_name)
    
    # get major title data
    mjr_title = df.iloc[0]['major_title']
    
    # get info_type data
    info_type = df.iloc[0]['info_type']
    
    #get link data
    linked_page = df.iloc[0]['Link']
    
    # make file with full name
    mk_file_full_name = college_name + "_" +  mjr_title + "_" + info_type + "_info" + ".txt"
    
    # dataframe에서 데이터 추출하고 파일명으로 저장하기
    md_file = open(mk_file_full_name, 'w')
    # Yale_African American Studies_general_keywords_info.txt
    
    # 링크데이터로 크롤링하기
    get_linked_data = linked_search_result(linked_page)
    
    # 데이터 클린징 후 리스트로 만들기
    cleaned_data = cleaning_data(str(get_linked_data))
    
    # 리스트를 문자열로 변환
    str_data = " ".join(cleaned_data)
    
    # 생성된 파일에 결과데이터 저장하기
    md_file.write(str_data)
    
    # 파일 닫기
    md_file.close()
    
    return

In [69]:
college_name = 'Yale'
file_name = './college_datasets/Yale/Yale 교수 전공 및 general 모든 정보 link.csv'
result = input_name_of_college(college_name, file_name)
result

/Users/kimkwangil/opt/anaconda3/envs/py37pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: use options instead of chrome_options
  # This is added back by InteractiveShellApp.init_path()


## auto-run code

In [83]:
# 자동으로 데이터 추출코드, 1)파일명 생성 2)링크접속 3)데이터크롤링 4)키워드로 저장
# file_name : 추출해야할 csv 파일 경로 및 파일명 지정
#     file_name = './college_datasets/Yale/Yale 교수 전공 및 general 모든 정보 link.csv'

# college_name : 'Yale'
# file_name =: './college_datasets/Yale/Yale 교수 전공 및 general 모든 정보 link.csv'
def input_name_of_college(college_name, file_name):
    # import file to dataframe
    df = pd.read_csv(file_name)
    
    i = 0
    for i in range(len(df.index)):
        # get major title data
        mjr_title = df.iloc[i]['major_title']

        # get info_type data
        info_type = df.iloc[i]['info_type']

        #get link data
        linked_page = df.iloc[i]['Link']

        # make file with full name
        mk_file_full_name = college_name + "_" +  mjr_title + "_" + info_type + "_info" + ".txt"

        # dataframe에서 데이터 추출하고 파일명으로 저장하기
        md_file = open(mk_file_full_name, 'w')
        # Yale_African American Studies_general_keywords_info.txt

        # 링크데이터로 크롤링하기
        try:
            get_linked_data = linked_search_result(linked_page)
            
            # 데이터 클린징 후 리스트로 만들기
            cleaned_data = cleaning_data(str(get_linked_data))

            # 리스트를 문자열로 변환
            str_data = " ".join(cleaned_data)

            # 생성된 파일에 결과데이터 저장하기
            md_file.write(str_data)

            # 파일 닫기
            md_file.close()

            # add numb
            i += 1
            
        except FileNotFoundError:
            md_file.close()

    return

In [84]:
college_name = 'Yale'
file_name = './college_datasets/Yale/Yale 교수 전공 및 general 모든 정보 link.csv'
result = input_name_of_college(college_name, file_name)
result

/Users/kimkwangil/opt/anaconda3/envs/py37pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: use options instead of chrome_options
  # This is added back by InteractiveShellApp.init_path()
